In [3]:
import csv

In [10]:
# check for missing columns

all_rows = []
header = None

input_file = "N5_kanji.csv"

with open(input_file, "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f)
    header = next(reader)
        
    for line_num, row in enumerate(reader, start=2):
        # Check if row is empty
        if not row or all(cell.strip() == "" for cell in row):
            print(f"[INFO] {input_file}, line {line_num}: Empty row skipped")
            continue
        # Check if row has exactly 6 columns:
        if len(row) != 6:
            print(f"[WARNING] {input_file}, line {line_num}: Expected 6 columns, found {len(row)} => {row}")
        else:
            all_rows.append(row)

print(f"{input_file} check complete")

N5_kanji.csv check complete
